In [11]:
pip install httplib2

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pickle
import urllib.request
import requests
from os.path import join,isfile
from os import listdir
from tqdm import tqdm
import urllib
import multiprocessing as mp
import pandas as pd
import time
import datetime
import httplib2

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [ ]:
with open('qids.pickle', 'rb') as f:
    graph = pickle.load(f)

In [ ]:
len(graph)

In [ ]:
mini_graph = graph[:110000]
print(len(mini_graph))

In [2]:
file = r'C:\Users\Алексей Кушнер\Sberbank\Geography\new_csv_dump\entities_автономный город_Q1094397.csv'
def csv_to_list(file):
    df = pd.read_csv(file , encoding='utf8',sep = ';')
    lst = list(df['entity'])
    return lst

In [3]:
csv_to_list(file)

['Q1486', 'Q5823', 'Q5831']

In [4]:
my_file = open('Logging.txt', 'w')
my_file.close()

In [5]:
my_file = open('Troubles.txt', 'w')
my_file.close()

In [6]:
print(datetime.datetime.now().time())

23:13:29.097157


In [15]:
%%file request_file.py


import pickle
import urllib.request
import requests
from os.path import join,isfile
from os import listdir
import urllib
from fake_useragent import UserAgent
import datetime
import time

ua = UserAgent()#Прокси

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles



def request_func(ID, func = 0):
    if func>5:
        with open(f"Trouble.txt", "a" , encoding='utf-8') as file:
                    file.write(f'{ID} :  {datetime.datetime.now().time()}\n')
    if f'D:\\dump\\{ID}.ttl' in counter_for_files('D:\\dump'):
        return None
    try:
        headers = {
        'User-Agent': ua.random
            } 
        with requests.get(f"https://www.wikidata.org/wiki/Special:EntityData/{ID}.ttl", 
                          stream=True, proxies=urllib.request.getproxies(), headers=headers) as r:
            if r.status_code != 200:
                with open(f"Logging.txt", "a" , encoding='utf-8') as file:
                    file.write(f'{ID} :  {r.status_code}\n')
                return None
            with open(f"D://dump//{ID}.ttl", "w" , encoding='utf-8') as file:
                file.write(r.text)
            return None
    except:
        time.sleep(3)
        return request_func(ID, func + 1)
        

Overwriting request_file.py


In [17]:
import request_file
pool = mp.Pool(processes=mp.cpu_count())
for file in counter_for_files('new_csv_dump'):
    file_name = file.split('\\')[-1].split('.')[0]
    start = time.time()
    mini_graph = csv_to_list(file)
    results = pool.map(request_file.request_func, mini_graph)
    end = time.time()
    print(f'Обработка {file_name} окончена , заняло {(end-start)/60} минут\n')

Обработка entities_автономный город_Q1094397 окончена , заняло 0.03889252742131551 минут

Обработка entities_административный город Республики Ирландия_Q13455645 окончена , заняло 0.03975118796030681 минут

Обработка entities_административный центр графства_Q1357964 окончена , заняло 0.1695544441541036 минут

Обработка entities_большой город в США_Q1093829 окончена , заняло 12.495049281915028 минут

Обработка entities_бывшая столица_Q27554677 окончена , заняло 0.04608984788258871 минут

Обработка entities_второй по величине город_Q50330360 окончена , заняло 0.040929277737935386 минут

Обработка entities_ганзейский город_Q707813 окончена , заняло 0.609826672077179 минут

Обработка entities_глобальный город_Q208511 окончена , заняло 0.03672715028127035 минут

Обработка entities_город Аргентины_Q5770918 окончена , заняло 0.1109410802523295 минут

Обработка entities_город без автомобилей_Q581830 окончена , заняло 0.041469649473826094 минут

Обработка entities_город в Альберте_Q55440238 око


Обработка entities_тхетсабаннакхон_Q15141632 окончена , заняло 0.09544488986333212 минут

Обработка entities_укреплённый город_Q677678 окончена , заняло 0.05604421297709147 минут

Обработка entities_университетский город_Q1187811 окончена , заняло 0.19609354734420775 минут

Обработка entities_уставный город Австрии_Q262882 окончена , заняло 0.06070362726847331 минут

Обработка entities_федеральный город_Q537250 окончена , заняло 0.03200826644897461 минут

Обработка entities_центральный город_Q2235075 окончена , заняло 0.030809982617696127 минут



In [ ]:
for file in counter_for_files('new_csv_dump')[0:2]:
    mini_graph = csv_to_list(file)
    if f'D://dump//{mini_graph[0]}.ttl' in counter_for_files('D:\\dump'):
        print(1)
    else :
        print(0)
    print(mini_graph[0])